<a href="https://colab.research.google.com/github/fernandodelaiglesia/cajondesastre/blob/master/AmplitudeAmplification_ProjectQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantum Amplitude Amplification with ProjectQ

This notebook provide the usage examples developed in

https://medium.com/@fernando.delaiglesia/quantum-amplitude-amplification-algorithm-qaa-in-projectq-7d675ddd89f1

The key reference material is:


*   The quick  [Wikipedia article](https://en.wikipedia.org/wiki/Amplitude_amplification), were you can find more references therein
*   The complete paper by G. Brassard, P. Hoyer, M. Mosca, A. Tapp (2000) Quantum Amplitude Amplification and Estimation https://arxiv.org/abs/quant-ph/0005055


## Preliminary notes

This notebook is created assuming that the Quantum Amplitude Amplification (QAA) gate is still in the develop branch, and therefore you need to clone the ProjectQ github repo to be able to use it.

If you use this notebook after QAA is included in the python package you don't need to clone the repo and add the path to the sys.path, but just to install projectq with:

`!pip install projectq`

In [1]:
! git clone https://github.com/ProjectQ-Framework/ProjectQ.git

Cloning into 'ProjectQ'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 2004 (delta 0), reused 0 (delta 0), pack-reused 2000
Receiving objects: 100% (2004/2004), 882.53 KiB | 3.22 MiB/s, done.
Resolving deltas: 100% (1468/1468), done.


In [0]:
import sys
sys.path.append('/content/ProjectQ/')

Import the requred libs

In [0]:
from projectq import MainEngine
from projectq.backends import Simulator
from projectq.ops import (X, H, Ry, All, Measure)
from projectq.meta import Loop, Control, Compute, Uncompute
from projectq.ops import QAA

import math

# Grover Algorithm

This example presents a especial case of QAA that is the famous Grover algorithm.

The Oracle selects the state `|1010101>` , and the initial algorithm applied on the `|0>` state is the Hadamard that creates a uniform superposition of all the states in the Hilbert space

In [0]:
def hache_algorithm(eng, qreg):
    All(H) | qreg


def simple_oracle(eng, system_q, control):
    # This oracle selects the state |1010101> as the one marked
    with Compute(eng):
        All(X) | system_q[1::2]
    with Control(eng, system_q):
        X | control
    Uncompute(eng)

Here we use QAA to obtain the selected state with high probability

In [0]:
eng = MainEngine()

n = 7
system_qubits = eng.allocate_qureg(n)

# Prepare the control qubit in the |-> state
control = eng.allocate_qubit()
X | control
H | control

# Creates the initial state form the Algorithm
hache_algorithm(eng, system_qubits)

# The correct number of iterations is related to the initial
# amplitude of the selected state after the execution of the
# Algorithm. For Grover that means an equal superposition of
# all the states, therefore the amplitude is 1/sqrt(2**7)

num_it = int(math.pi/4.* math.sqrt(2**n))

# Apply Quantum Amplitude Amplification the correct number of times
with Loop(eng, num_it):
    QAA(hache_algorithm, simple_oracle) | (system_qubits, control)

# If we measure the system we would find the selected state with
# high probability

All(Measure) | system_qubits
H | control
Measure | control
result = [int(q) for q in system_qubits]
control_result = int(control)

print (result)

eng.flush()


(Note: This is the (slow) Python simulator.)
[1, 0, 1, 0, 1, 0, 1]


# QAA for a complex algorithm and oracle

For a more general aplication of QAA we can use an oracle that selects a subspace from the Hilbert space starting form a general algorithm creating a superposition that is not uniform.



In [0]:
def complex_algorithm(eng, qreg):
    All(H) | qreg
    with Control(eng, qreg[0]):
        All(X) | qreg[1:]
    All(Ry(math.pi / 4)) | qreg[1:]
    with Control(eng, qreg[-1]):
        All(X) | qreg[1:-1]


def complex_oracle(eng, system_q, control):
    # This oracle selects the subspace |000000>+|111111> as the good one
    with Compute(eng):
        with Control(eng, system_q[0]):
            All(X) | system_q[1:]
        H | system_q[0]
        All(X) | system_q

    with Control(eng, system_q):
        X | control

    Uncompute(eng)


## If we know the initial amplitude of the selected subspace (good subspace)

For the first example let's supose that we know the probability/amplitude of the good subspace. In this case we can know easily the number of iterarios to run the QAA to obtain the selected subspace with high probability. From our simulator this is very easy to obtain by cheating and getting the probability of the states

In [6]:
eng = MainEngine()

system_qubits = eng.allocate_qureg(6)

# Prepare the control qubit in the |-> state
control = eng.allocate_qubit()
X | control
H | control

# Creates the initial state form the Algorithm
complex_algorithm(eng, system_qubits)

# Get the probabilty of getting the marked state before the QAA
# to calculate the number of iterations
eng.flush()
prob000000 = eng.backend.get_probability('000000', system_qubits)
prob111111 = eng.backend.get_probability('111111', system_qubits)

total_amp_before = math.sqrt(prob000000 + prob111111)
theta_before = math.asin(total_amp_before)

# The number of iterations to run QAA is related to the probability
# of the good subspace after applying the algorithm for the first
# time to the |0> state

num_it = int(math.pi / (4. * theta_before) + 1)

# Apply Quantum Amplitude Amplification the correct number of times

with Loop(eng, num_it):
    QAA(complex_algorithm, complex_oracle) | (system_qubits, control)

# If we measure the system now we would obtain a result in the
# good subspace with high probability

All(Measure) | system_qubits
H | control
Measure | control
result = [int(q) for q in system_qubits]
control_result = int(control)

print (result)

eng.flush()


(Note: This is the (slow) Python simulator.)
[1, 1, 1, 1, 1, 1]
